<a href="https://colab.research.google.com/github/LegendX369/Ai_Project_06/blob/main/kshitizA.i.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import libraries and datas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

dataset = pd.read_excel("/content/HousePricePrediction.xlsx")

print(dataset.head(5))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
#Data Preprocessing
obj = (dataset.dtypes == 'object')
object_cols = list(obj[obj].index)
print("Categorical variables:",len(object_cols))

int_ = (dataset.dtypes == 'int')
num_cols = list(int_[int_].index)
print("Integer variables:",len(num_cols))

fl = (dataset.dtypes == 'float')
fl_cols = list(fl[fl].index)
print("Float variables:",len(fl_cols))

In [ ]:
# Exploratory Data Analysis
# Numerical correlations heatmap
numerical_dataset = dataset.select_dtypes(include=['number'])
plt.figure(figsize=(12, 6))
sns.heatmap(numerical_dataset.corr(), cmap='BrBG', fmt='.2f', linewidths=2, annot=True)
plt.title('Correlation Heatmap')
plt.show()
# Unique values in categorical features (horizontal bars for values on the side)
object_cols = dataset.select_dtypes(include=['object']).columns
unique_values = [dataset[col].nunique() for col in object_cols]
plt.figure(figsize=(10, 6))
sns.barplot(x=unique_values, y=object_cols)
plt.title('Unique Values in Categorical Features')
plt.xlabel('Count')
plt.show()

In [ ]:
# Get categorical columns
object_cols = dataset.select_dtypes(include=['object']).columns
# Determine grid size based on number of columns
n_cols = len(object_cols)
cols = 4
rows = (n_cols + cols - 1) // cols  # Ceiling division
plt.figure(figsize=(18, 6 * rows))  # Adjust height dynamically
plt.suptitle('Categorical Features: Distribution', fontsize=16)
index = 1
for col in object_cols:
    y = dataset[col].value_counts()
    plt.subplot(rows, cols, index)
    sns.barplot(x=list(y.index), y=y)
    plt.title(col)
    plt.xticks(rotation=90)
    index += 1
plt.tight_layout()  # Removes extra blank space
plt.show()

In [ ]:
dataset.drop(['Id'],
             axis=1,
             inplace=True)
dataset['SalePrice'] = dataset['SalePrice'].fillna(
  dataset['SalePrice'].mean())
new_dataset = dataset.dropna()
new_dataset.isnull().sum()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

s = (new_dataset.dtypes == 'object')
object_cols = list(s[s].index)
print("Categorical variables:")
print(object_cols)
print('No. of. categorical features: ',
      len(object_cols))

In [ ]:
OH_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
OH_cols = pd.DataFrame(OH_encoder.fit_transform(new_dataset[object_cols]))
OH_cols.index = new_dataset.index
OH_cols.columns = OH_encoder.get_feature_names_out()
df_final = new_dataset.drop(object_cols, axis=1)
df_final = pd.concat([df_final, OH_cols], axis=1)

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

X = df_final.drop(['SalePrice'], axis=1)
Y = df_final['SalePrice']

X_train, X_valid, Y_train, Y_valid = train_test_split(
    X, Y, train_size=0.8, test_size=0.2, random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model_RFR = RandomForestRegressor(n_estimators=10)
model_RFR.fit(X_train, Y_train)
Y_pred = model_RFR.predict(X_valid)

mean_absolute_percentage_error(Y_valid, Y_pred)

In [ ]:
from sklearn.linear_model import LinearRegression

model_LR = LinearRegression()
model_LR.fit(X_train, Y_train)
Y_pred = model_LR.predict(X_valid)

print(mean_absolute_percentage_error(Y_valid, Y_pred))